Hotel Recommendation System

Bizim yaklaşımımız müşteri yorumlarına ve derecelendirmelerine dayalı bir öneri sistemi oluşturmaktır.Kullanıcı puanlarına göre bir öneri sistemi oluşturacağım için burada doğal dil işleme kullanacağım. 

In [13]:
# Kullanacağım dataset.

import kagglehub

# Download latest version
path = kagglehub.dataset_download("jiashenliu/515k-hotel-reviews-data-in-europe")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Asus\.cache\kagglehub\datasets\jiashenliu\515k-hotel-reviews-data-in-europe\versions\1


In [14]:
# Veri setini okuyalım.

import pandas as pd
df=pd.read_csv("C:/Users/Asus/.cache/kagglehub/datasets/jiashenliu/515k-hotel-reviews-data-in-europe/versions/1/Hotel_Reviews.csv")
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [ ]:
# Hotel_Address sütununun son kelimesi ülke adı oluyor.Ülke adını ayrı bir sütun haline getirelim.
# Aynı zamanda United Kingdom'u UK olarak değiştirelim.(Tek kelime olsun)

df['Hotel_Address'] = df['Hotel_Address'].str.replace("United Kingdom", "UK")
df['Country'] = df['Hotel_Address'].str.split(' ').str[-1]
print(df['Country'].value_counts())

Country
UK             262301
Spain           60149
France          59928
Netherlands     57214
Austria         38939
Italy           37207
Name: count, dtype: int64


In [16]:
# DataFrame'deki güncel satır ve sütun sayısını görelim.
df.shape

(515738, 18)

In [17]:
# Kaç tane tekrar eden veri var?
print(df.duplicated().sum())  

526


In [18]:
# Veri setinde 526 tane tekrar eden satır var.Bunları kaldıralım.
# Ardından df.shape ile temizleme sonrası DataFrame'deki güncel satır ve sütun sayısını görelim.

df = df.drop_duplicates()
print(f"Yeni veri seti boyutu: {df.shape}")

Yeni veri seti boyutu: (515212, 18)


In [19]:
# Eksik/hatalı veriler var mı?
print(df.isnull().sum())

Hotel_Address                                    0
Additional_Number_of_Scoring                     0
Review_Date                                      0
Average_Score                                    0
Hotel_Name                                       0
Reviewer_Nationality                             0
Negative_Review                                  0
Review_Total_Negative_Word_Counts                0
Total_Number_of_Reviews                          0
Positive_Review                                  0
Review_Total_Positive_Word_Counts                0
Total_Number_of_Reviews_Reviewer_Has_Given       0
Reviewer_Score                                   0
Tags                                             0
days_since_review                                0
lat                                           3268
lng                                           3268
Country                                          0
dtype: int64


In [20]:
# lat ve lng sütunlarında eksik veriler var.Bu sütunları çıkaracağım.
# Modelin doğruluğunu etkileyip etkilemediğini daha sonra test edeceğim.Gerekirse geri ekleyeceğim.

df.drop(['lat', 'lng'], axis=1, inplace=True)

# Verinin yeni halini kontrol edelim
print(df.shape)


(515212, 16)


In [21]:
print(df.columns)

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'Country'],
      dtype='object')


In [22]:
# Dataset'te 6 farklı ülke var.
print(df['Country'].unique())

['Netherlands' 'UK' 'France' 'Spain' 'Italy' 'Austria']


Feature selection(özellik seçimi), genellikle modelin daha hızlı çalışmasını sağlar ve aşırı öğrenmeyi (overfitting) engeller. Bu yüzden, özellikle çok sayıda özellik olduğunda önemlidir.

In [25]:
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,Country
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,Netherlands
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,Netherlands
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,Netherlands
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,Netherlands
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,Netherlands


In [ ]:
# Review_Date sütunu üzerinde işlemler yapacağım.
# datetime formatına dönüştürülüyor. errors='coerce' parametresi kullanıldığında, geçersiz veya hatalı tarih formatları olan hücreler
# NaT (Not a Time) değerine dönüştürülür. 
# Bu sayede veri setindeki tarih formatları tutarsız olsa bile hatalar düzgün şekilde ele alınır.

df['Review_Date'] = pd.to_datetime(df['Review_Date'], errors='coerce')
print(df['Review_Date'].head())


0   2017-08-03
1   2017-08-03
2   2017-07-31
3   2017-07-31
4   2017-07-24
Name: Review_Date, dtype: datetime64[ns]


In [39]:
# Kullanıcıların yıl ve ay üzerinden filtreleme yapabilmesini sağlayacağım.
#Değerlendirme yılı ve değerlendirme ayı şeklinde sütunlar oluşturalım.
df['Review_Year']=df['Review_Date'].dt.year

#Eğer kullanıcılar ayları sayısal olarak değil de isim olarak görsün.
df['Review_Month_Name'] = df['Review_Date'].dt.strftime('%B')

In [40]:
df.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,...,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,Country,Year,Month,Review_Year,Review_Month,Review_Month_Name
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,...,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,Netherlands,2017,8,2017,8,August
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-08-03,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,...,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,Netherlands,2017,8,2017,8,August
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,...,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,Netherlands,2017,7,2017,7,July
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-31,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,...,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,Netherlands,2017,7,2017,7,July
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,2017-07-24,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,...,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,Netherlands,2017,7,2017,7,July


# STREAMLİT KISMI.Sonraki aşamalarda.

# Streamlit widget'ları ile yıl ve ay seçimi
selected_year = st.selectbox('Yıl Seçin', df['Year'].unique())
selected_month = st.selectbox('Ay Seçin', df[df['Year'] == selected_year]['Month'].unique())

# Seçilen yıl ve ay için veri filtreleme
filtered_data = df[(df['Year'] == selected_year) & (df['Month'] == selected_month)]

# Kullanıcıya filtrelenmiş veriyi gösterme
st.write(filtered_data)